Import dependencies

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from extractor import fetch_website_links, fetch_website_contents
import os
import json

load_dotenv()

Define Open AI keys

In [ ]:
OPENROUTER_API_KEY= os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL= os.getenv("OPENROUTER_BASE_URL")



Rail guards check

In [ ]:
if OPENROUTER_API_KEY:
    print("API_KEY found")
else:
    print("No API_KEY found")
    exit(1)

Define OpenAI client library

In [ ]:
client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)

Define system and user prompt 

In [ ]:

system_one_shot_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def user_prompts(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt
    

Send request to open ai model

In [ ]:
def select_relevant_link(url):
  response = client.chat.completions.create(
          model="openai/gpt-3.5-turbo",
          messages=[
                {
                  "role": "system",
                  "content": system_one_shot_prompt
                },
                {
                    "role": "user",
                    "content": user_prompts(url)
                }
              ],
          response_format= {"type": "json_object"}

      )
  result = response.choices[0].message.content
  links = json.loads(result)

  print(f"Found {len(links['links'])} relevant links")
  return links


In [ ]:
select_relevant_link("https://huggingface.co")

Bronchure Generator


In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_link(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
fetch_page_and_all_relevant_links("https://huggingface.co")

In [ ]:
brochure_system_prompt = """
You are a competent bronchure assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
You should also create the bronchure based on the type of company it is. for example if the company is a personal portfolio of a developer then the bronchure should focus on the skills and projects of the developer.
Do not add content that is not present in the provided text and make sure to only use the information given.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [ ]:
def create_brochure(company_name, url):
    stream_response = client.chat.completions.create(
        model="openai/gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream_response:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


In [ ]:
create_brochure("My-Portfolio", "https://jessica-ovabor-portfolio.vercel.app/")